In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from utils.transform import Transform
from riskfuel_test import riskfuel_test
import torch
from utils.model import PutNet

/home/vigu/pyenvs/vanilla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = Transform(use_boxcox=True)

In [3]:
df = pd.read_csv("dataset/training_data.csv")

In [4]:
x = torch.Tensor(transform.transform_x(df[["S", "K", "T", "r", "sigma"]].to_numpy()))
y = torch.Tensor(df[["value"]].to_numpy()).flatten()

In [5]:
mm = PutNet()
# mm.load_state_dict(torch.load("simple-model.pt"))
mm.load_state_dict(torch.load("models/model_1900000_train_42300_epoch_192_neurons_boxcox_scaler.pt"))
mm.eval()  # evaluation mode

PutNet(
  (layers): ModuleList(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
  )
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [6]:
y_hat = torch.Tensor(transform.inverse_transform_y(mm(x).detach().cpu().numpy()))

In [7]:
diff_vals, diff_ids = torch.sort(torch.abs(y - y_hat), descending=True)

In [8]:
new_x = x[diff_ids[:1000000]]
new_y = y[diff_ids[:1000000]].reshape(-1, 1)

In [9]:
xy = np.append(new_x, new_y, axis=1)
xy_df = pd.DataFrame(xy, columns=["S", "K", "T", "r", "sigma", "value"])
xy_df.to_csv("dataset/training_data.csv")